# No. 1

In [2]:
import firebase_admin
from firebase_admin import credentials, storage
import pandas as pd

# Fungsi untuk ekstraksi data dari CSV
def extract_data_csv(csv):
    df = pd.read_csv(csv)
    return df

# Fungsi untuk transformasi data (menghapus data duplikat dan tidak lengkap)
def transform_data(df):
    df_cleaned = df.drop_duplicates()  # Menghapus data duplikat
    df_cleaned = df_cleaned.dropna()   # Menghapus data yang tidak lengkap
    
    # Menghapus baris dengan nilai '-' pada kolom 'age'
    df_cleaned = df_cleaned[df_cleaned['age'] != "-"]

    # Menghapus baris dengan nilai '-' pada kolom 'favorite_front_end'
    df_cleaned = df_cleaned[df_cleaned['favorite_front_end'] != "-"]
    
    df_cleaned = df_cleaned.reset_index(drop=True)
    return df_cleaned

# Fungsi untuk load ke Google Storage
def load_data_to_firebase():
    try:
        # Cek apakah aplikasi Firebase sudah diinisialisasi sebelumnya
        if not firebase_admin._apps:
            # Jika belum, inisialisasi aplikasi Firebase
            cred = credentials.Certificate("accountKey.json")
            firebase_admin.initialize_app(cred, {
                "storageBucket": "de-in-cloud-nuri-hidayatuloh.appspot.com"
            })
        
        bucket = storage.bucket()
        
        csv = "survey_cleaned.csv"
        
        blob = bucket.blob(csv)
        blob.upload_from_filename(csv, content_type='text/csv')
        
        print("Load Data Berhasil")
    except Exception as e:
        print("Error:", e)

# Jalankan fungsi-fungsi secara berurutan
if __name__ == "__main__":
    try:
        # Ekstraksi data dari CSV
        csv_data = "survey.csv"
        extracted_df = extract_data_csv(csv_data)
        
        # Transformasi data
        transformed_df = transform_data(extracted_df)
        
        # Load data yang telah ditransformasi ke Google Storage
        load_data_to_firebase()
    except Exception as e:
        print("Error:", e)

Load Data Berhasil


In [3]:
# Simpan DataFrame yang telah ditransformasi ke file CSV secara lokal
csv_filename = "survey.csv"
transformed_df = transform_data(extract_data_csv(csv_filename))
csv_filename_local = "survey_cleaned.csv"
transformed_df.to_csv(csv_filename_local, index=False)

# No. 2

In [4]:
import firebase_admin
from firebase_admin import credentials, storage
import pandas as pd
import requests

# Fungsi untuk ekstraksi data dari JSON URL
def extract_data_json(json_url):
    response = requests.get(json_url)
    data = response.json()
    df = pd.DataFrame(data)
    return df

# Fungsi untuk transformasi data (mengambil data transaksi dengan metode pembayaran kartu kredit dan status success)
def transform_data(df):
    df_transformed = df[(df['payment_method'] == 'credit card') & (df['status'] == 'success')]
    return df_transformed

# Fungsi untuk load ke Google Storage
def load_data_to_firebase(df, storage_bucket, filename):
    try:
        # Cek apakah aplikasi Firebase sudah diinisialisasi sebelumnya
        if not firebase_admin._apps:
            # Jika belum, inisialisasi aplikasi Firebase
            cred = credentials.Certificate("accountKey.json")
            firebase_admin.initialize_app(cred, {
                "storageBucket": storage_bucket
            })
        
        bucket = storage.bucket()
        
        blob = bucket.blob(filename)
        blob.upload_from_string(df.to_csv(index=False), content_type='text/csv')
        
        print("Load Data Berhasil")
    except Exception as e:
        print("Error:", e)

# Jalankan fungsi-fungsi secara berurutan
if __name__ == "__main__":
    try:
        # Ekstraksi data dari JSON URL
        json_url = "https://gist.githubusercontent.com/nadirbslmh/8fc9cc6cd5cbaaf5cbff63b090fb497e/raw/a7bf3e1edab88b04314a40a9de3ed744bc86d0e9/ecommerce.json"
        df = extract_data_json(json_url)
        
        # Tampilkan DataFrame yang telah diextract
        print("DataFrame setelah ekstraksi:")
        print(df.head())  # Tampilkan lima baris pertama DataFrame
        
        # Transformasi data
        transformed_df = transform_data(df)
        
        # Tampilkan DataFrame yang telah ditransformasi
        print("\nDataFrame setelah transformasi:")
        print(transformed_df.head())  # Tampilkan lima baris pertama DataFrame yang telah ditransformasi
        
        # Simpan DataFrame yang telah ditransformasi ke file CSV secara lokal
        csv_filename_local = "ecommerce_transactions.csv"
        transformed_df.to_csv(csv_filename_local, index=False)
        print("\nData disimpan sebagai", csv_filename_local)
        
        # Load data yang telah ditransformasi ke Google Storage
        storage_bucket = "de-in-cloud-nuri-hidayatuloh.appspot.com"
        filename = "ecommerce_transactions.csv"
        load_data_to_firebase(transformed_df, storage_bucket, filename)
    except Exception as e:
        print("Error:", e)

DataFrame setelah ekstraksi:
   transaction_id transaction_date   customer_name  transaction_amount  \
0               1       01-11-2023        John Doe                50.0   
1               2       01-11-2023      Jane Smith               120.5   
2               3       02-11-2023     Bob Johnson               800.0   
3               4       03-11-2023  Alice Williams                30.0   
4               5       03-11-2023   Charlie Brown               150.0   

  item_category              item_name payment_method   status  
0          book       The Great Gatsby    credit card  success  
1        gaming          PlayStation 5           cash  success  
2    technology                 Laptop    credit card   failed  
3       kitchen           Coffee Maker       e wallet  success  
4          book  To Kill a Mockingbird    credit card  success  

DataFrame setelah transformasi:
    transaction_id transaction_date   customer_name  transaction_amount  \
0                1       01-